In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
# setting device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# dataloader
class lstmdataset(Dataset):
    def __init__(self):
        # Reshaping X for the LSTM 
        data = np.load('../parsed_mimetics/gcn_train_data.npy')
        labels = np.load('../parsed_mimetics/gcn_train_label.npy')
        data = torch.DoubleTensor(data)
        self.data = data.transpose(1, 2).reshape(679, 300, -1)
        self.labels = torch.DoubleTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]
        return X, y

In [4]:
# The LSTM Model

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

In [5]:
dataset = lstmdataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
batch_size = 4
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Parameters
input_size = 150
hidden_size = 64
num_classes = 50
num_epochs = 50

model = LSTMClassifier(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Train the model

train_acc = []
test_acc = []

for epoch in tqdm(range(num_epochs)):
    train_loss = 0
    num_batches = 0
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        num_batches += 1
    train_loss /= num_batches

    # Evaluate on test set
    model.eval()
    test_accuracy = 0
    num_samples = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs.float())
            _, predicted = torch.max(outputs.data, 1)
            num_samples += labels.size(0)
            test_accuracy += (predicted == labels).sum().float().item()
        test_accuracy /= num_samples
    
    train_acc.append(train_loss)
    test_acc.append(test_accuracy)

100%|██████████| 50/50 [00:40<00:00,  1.23it/s]


In [8]:
print(train_acc)
print(test_acc)

[3.9121187101392185, 3.8463788891539856, 3.80570790346931, 3.773210455389584, 3.75249044509495, 3.742672688820783, 3.7250477373600006, 3.7151656185879425, 3.705537329701816, 3.6902522146701813, 3.673467958674711, 3.6602681699921105, 3.6546169151278103, 3.6552964939790615, 3.6702098232858322, 3.6489857926088223, 3.6466777587638184, 3.6514411200495327, 3.6367715421844933, 3.6149955058799073, 3.6051219701766968, 3.5953789037816666, 3.5970523532699135, 3.603326271562015, 3.585514221121283, 3.5692016499883987, 3.589064163320205, 3.582719922065735, 3.570029143024893, 3.5610341718968224, 3.5567642976255978, 3.5732632875442505, 3.553727446233525, 3.5516606201143825, 3.5576677629176308, 3.5471617895014145, 3.527339765254189, 3.517997268368216, 3.5135119636269176, 3.5137204808347366, 3.5198181122541428, 3.5173570820513893, 3.5485446049886593, 3.5266787585090187, 3.513462048243074, 3.5036321483990727, 3.4939570681137195, 3.4991484468474106, 3.4916244087850346, 3.4824221546159073]
[0.0, 0.02941176